In [11]:
import pandas as pd
from functions import normalize_name, add_zeroes

Load the data

In [12]:
df = pd.read_csv('C:/Users/HP/Desktop/Traineeship/data/Elenco dei codici e delle denominazioni delle unit_ territoriali.csv', sep = ';')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7896 entries, 0 to 7895
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Codice Ripartizione geografica  7896 non-null   int64 
 1   Codice Regione                  7896 non-null   int64 
 2   Codice Provincia (Storico)      7896 non-null   int64 
 3   Codice Provincia/Uts            7896 non-null   int64 
 4   Codice Comune (alfanumerico)    7896 non-null   int64 
 5   Codice Comune (numerico)        7896 non-null   int64 
 6   Comune                          7895 non-null   object
 7   Comune (dizione italiana)       7895 non-null   object
 8   Comune (dizione straniera)      124 non-null    object
 9   Ripartizione geografica         7896 non-null   object
 10  Regione                         7896 non-null   object
 11  Provincia/Uts                   7896 non-null   object
 12  Flag tipo Uts                   7896 non-null   

Translate and clean

In [13]:
dict = {
    'Comune':'mun_name',
    'Provincia/Uts' : 'prov_name',
    'Codice Comune (alfanumerico)':'mun_istat',
    'Codice Provincia (Storico)' : 'prov_istat',
    'Regione' : 'reg_name'
}

df = df.rename(columns = dict)

In [14]:
# Order columns
order = ['mun_name','prov_name','mun_istat','prov_istat','reg_name']

df = df[order]

In [15]:
# Add missing heading 0s to Istat codes
add_zeroes(df, ['mun_istat'], 6)

df['mun_istat'] = df['mun_istat'].astype('object')

# Add missing heading 0s to provincial Istat codes
add_zeroes(df, ['prov_istat'], 3)

df['prov_istat'] = df['prov_istat'].astype('object')


# Normalize municipalities, province, and region names
col_to_norm = ['mun_name','prov_name','reg_name']

for col in col_to_norm:
    df[col + '_norm'] = df[col].apply(normalize_name).astype('object')

In [16]:
# Check for duplicates
df[df['mun_name_norm'].duplicated(keep=False)].sort_values('mun_name_norm')

,mun_name,prov_name,mun_istat,prov_istat,reg_name,mun_name_norm,prov_name_norm,reg_name_norm
1894,Castro,Bergamo,016065,016,Lombardia,castro,bergamo,lombardia
6057,Castro,Lecce,075096,075,Puglia,castro,lecce,puglia
1552,Livo,Como,013130,013,Lombardia,livo,como,lombardia
2822,Livo,Trento,022106,022,Trentino-Alto Adige/Südtirol,livo,trento,trentinoalto adigesudtirol
6159,Paterno,Potenza,076100,076,Basilicata,paterno,potenza,basilicata
6850,Paternò,Catania,087033,087,Sicilia,paterno,catania,sicilia
1577,Peglio,Como,013178,013,Lombardia,peglio,como,lombardia
3983,Peglio,Pesaro e Urbino,041041,041,Marche,peglio,pesaro e urbino,marche
230,Samone,Torino,001235,001,Piemonte,samone,torino,piemonte
2863,Samone,Trento,022165,022,Trentino-Alto Adige/Südtirol,samone,trento,trentinoalto adigesudtirol


In [17]:
# Create a key [mun_name,region] to uniquely identify duplicates
df['mun_key'] = df['mun_name_norm'] + '_' + df['reg_name_norm']

In [18]:
# Check for duplicates
df[df['mun_key'].duplicated(keep=False)].sort_values('mun_key')

,mun_name,prov_name,mun_istat,prov_istat,reg_name,mun_name_norm,prov_name_norm,reg_name_norm,mun_key


Save the data

In [19]:
df.to_parquet('datasets/mun_istat_codes.parquet', index = False)